In [1]:
# app.py
from flask import Flask, jsonify
import requests
import gtfs_realtime_pb2  # Import the generated file for GTFS-Realtime parsing

In [2]:
app = Flask(__name__)

# URL for MTA's GTFS-Realtime data (example for A, C, E lines)
MTA_FEED_URL = 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-ace'

@app.route('/service-alerts', methods=['GET'])
def get_service_alerts():
    # Initialize the GTFS-Realtime FeedMessage
    feed = gtfs_realtime_pb2.FeedMessage()
    
    # Fetch GTFS-Realtime data from the MTA
    try:
        response = requests.get(MTA_FEED_URL)
        response.raise_for_status()
    except requests.RequestException as e:
        return jsonify({"error": "Failed to fetch MTA data", "details": str(e)}), 500

    # Parse the response content
    feed.ParseFromString(response.content)
    
    # Extract service alerts related to route changes or delays
    alerts = []
    for entity in feed.entity:
        if entity.HasField('alert'):
            alert = entity.alert
            # Check if alert text mentions delays or reroutes
            alert_text = alert.header_text.translation[0].text
            if "delay" in alert_text.lower() or "reroute" in alert_text.lower():
                alerts.append(alert_text)

    # Return the alerts as JSON
    if alerts:
        return jsonify({"alerts": alerts}), 200
    else:
        return jsonify({"alerts": "No route changes or delays detected"}), 200

# Run the Flask app on a custom port
if __name__ == '__main__':
    app.run(debug=True, port=5001)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/carlamandiola/.pyenv/versions/3.11.6/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/carlamandiola/.pyenv/versions/3.11.6/lib/python3.11/site-packages/traitlets/config/application.py", line 1052, in launch_instance
    app.initialize(argv)
  File "/Users/carlamandiola/.pyenv/versions/3.11.6/lib/python3.11/site-packages/traitlets/config/application.py", line 117, in inner
    return me

SystemExit: 1

/Users/carlamandiola/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
